In [6]:
import asset_asrl as ast
import numpy as np

vf = ast.VectorFunctions #module that contains types and functions
Args = vf.Arguments # base object


In [9]:
xvals = np.array([0,1,2,3,4,5])
lvals = np.ones((6))

X = Args(6) # A first class function thats takes any vector of size 6 and returns that vector

x0 = X[0]

type(X)

asset.VectorFunctions.Arguments

In [ ]:
xvals = np.array([1,2,3,4,5,6,7,8])

X = Args(8)

R = X.head(3)
V = X.segment(3,4)
t = X[6]
u = X[7]

## Equivalent to the Above
#R,V,t,u = X.tolist([ (0,3), (3,3), (6,1),(7,1) ])


A = X.segment3(2)
print(f"R:TYPE: {type(R)}\n")
print(f"V:TYPE: {type(V)}, V_values: {V(xvals)}\n")
print(f"V:TYPE: {type(A)}, V_values: {A(xvals)}\n")
print(type(A))
print(type(A(xvals)))

r1 = R[2]

print(r1(xvals))

R:TYPE: <class 'asset.VectorFunctions.Segment3'>

V:TYPE: <class 'asset.VectorFunctions.Segment'>, V_values: [1. 2. 3. 4.]

A:TYPE: <class 'asset.VectorFunctions.Segment3'>, A_values: [3. 4. 5.]

<class 'asset.VectorFunctions.Segment3'>
<class 'numpy.ndarray'>
[3.]


In [ ]:
xvals = np.array([1,2,3,4,5,6])

X = Args(6)

R = X.head(3)
V = X.tail(3)

RpV = R*2
RpV(xvals)

## RmV = R*V  # Throws and Error
## RdV = R/V  # Throws and Error

array([2., 4., 6.])

In [61]:
x0,x1,x2 = Args(3).tolist()

condition = x0<1.0

output_if_true = x1*2
output_if_false = x1+x2

func = vf.ifelse(condition,output_if_true,output_if_false)

In [62]:
print(func([0,  2,3]))  # prints [4.0]
print(func([1.5,2,3]))  # prints [5.0]

[4.]
[5.]


In [63]:
def RTNBasis():

        R,V = Args(6).tolist([(0,3),(3,3)])

        Rhat = R.normalized()
        Nhat = R.cross(V).normalized()
        That = Nhat.cross(R).normalized()

        return vf.stack(Rhat,That,Nhat)

In [64]:
type(RTNBasis)

function

In [65]:
oc = ast.OptimalControl

XVars = 6
UVars = 3
PVars = 0

XtU = oc.ODEArguments(XVars,UVars,PVars) # [ [R,V] , t, [U],[]]

In [ ]:
import asset_asrl as ast
import numpy as np

vf        = ast.VectorFunctions
oc        = ast.OptimalControl
Args      = vf.Arguments



In [100]:
XVars = 6
UVars = 3
PVars = 0

XtU = oc.ODEArguments(XVars,UVars,PVars) # [ [R,V] , t, [U],[]]



In [113]:
xvals = np.array([1,2,3,4,5,6])
type(XtU.XVec())

asset.VectorFunctions.Segment

In [1]:
import numpy as np
import asset_asrl as ast
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap ## PIP INSTALL Basemap if you dont have it

vf        = ast.VectorFunctions
oc        = ast.OptimalControl
Args      = vf.Arguments

############################################################################

g0      =  9.80665
Lstar   =  6378145           ## m   Radius of Earth
Tstar   =  961.0             ## sec Engine Burn Time
Mstar   =  301454.0          ## kgs Inital Mass of Rocket


Astar   =  Lstar/Tstar**2
Vstar   =  Lstar/Tstar
Rhostar =  Mstar/Lstar**3
Estar   =  Mstar*(Vstar**2)
Mustar  =  (Lstar**3)/(Tstar**2)
Fstar   =  Astar*Mstar
#############################################################################

mu      = 3.986012e14      /Mustar
Re      = 6378145          /Lstar
We      = 7.29211585e-5    *Tstar

RhoAir  = 1.225        /Rhostar
h_scale = 7200         /Lstar
g       = g0           /Astar


CD = .5
S  = 4*np.pi   /Lstar**2



TS = 628500      /Fstar
T1 = 1083100     /Fstar
T2 = 110094      /Fstar

IS = 283.33364   /Tstar
I1 = 301.68      /Tstar
I2 = 467.21       /Tstar

tS = 75.2        /Tstar
t1 = 261         /Tstar
t2 = 700         /Tstar


TMS = 19290      /Mstar
TM1 = 104380     /Mstar
TM2 = 19300      /Mstar
TMPay = 4164     /Mstar


PMS = 17010     /Mstar
PM1 = 95550     /Mstar
PM2 = 16820     /Mstar

SMS = TMS - PMS
SM1 = TM1 - PM1
SM2 = TM2 - PM2

T_phase1 = 6*TS + T1
T_phase2 = 3*TS + T1
T_phase3 = T1
T_phase4 = T2

mdot_phase1 = (6*TS/IS + T1/I1)/g
mdot_phase2 = (3*TS/IS + T1/I1)/g
mdot_phase3 = T1/(g*I1)
mdot_phase4 = T2/(g*I2)


tf_phase1 = tS
tf_phase2 = 2*tS
tf_phase3 = t1
tf_phase4 = t1+t2

m0_phase1 = 9*TMS + TM1 + TM2 + TMPay
mf_phase1 = m0_phase1 - 6*PMS - (tS/t1)*PM1

m0_phase2 = mf_phase1 - 6*SMS
mf_phase2 = m0_phase2 - 3*PMS - (tS/t1)*PM1

m0_phase3 = mf_phase2 - 3*SMS
mf_phase3 = m0_phase3 - (1 - 2*tS/t1)*PM1

m0_phase4 = mf_phase3 - SM1
mf_phase4 = m0_phase4 - PM2


#############################################################################
class RocketODE(oc.ODEBase):
    def __init__(self,T,mdot):
        ####################################################
        XtU  = oc.ODEArguments(7,3)

        R = XtU.XVec().head3()
        V = XtU.XVec().segment3(3)
        m = XtU.XVar(6)

        # We normalize the control direction in the dynamics so it doesnt have
        # to be done as a path constraint
        u = XtU.UVec().normalized()

        h       = R.norm() - Re
        rho     = RhoAir * vf.exp(-h / h_scale)
        Vr      = V + R.cross(np.array([0,0,We]))

        ## We cant let let Vr be exactly 0, derivative of norm of 0 vector is NAN
        ## Will handle this in inititial conditions
        D       = (-0.5*CD*S)*rho*(Vr*Vr.norm())

        Rdot    =  V
        Vdot    =  (-mu)*R.normalized_power3() + (T*u + D)/m

        ode = vf.stack(Rdot,Vdot,-mdot)
        ####################################################
        super().__init__(ode,7,3)

def TargetOrbit(at,et,it, Ot,Wt):
    R,V = Args(6).tolist([(0,3),(3,3)])

    r    = R.norm()
    v    = V.norm()

    #Angular momentum vector
    hvec = R.cross(V)

    #Node vector
    nvec = vf.cross([0,0,1],hvec)

    # Energy
    eps = 0.5*(v**2) - mu/r

    # Semi-major axis
    a =  -0.5*mu/eps

    evec = V.cross(hvec)/mu - R.normalized()
    #Eccentrcity
    e = evec.norm()

    #inclination
    i = vf.arccos(hvec.normalized()[2])

    #RAAN
    O = vf.arccos(nvec.normalized()[0])
    # Quadrant Check
    O = vf.ifelse(nvec[1]>0,O,2*np.pi -O)

    # Argument of periapse
    W = vf.arccos(nvec.normalized().dot(evec.normalized()))
    #QuadrantCheck
    W = vf.ifelse(evec[2]>0,W,2*np.pi-W)

    return vf.stack([a,e,i,O,W]) - np.array([at,et,it,Ot,Wt])

###############################################################################

def Plot(Phase1,Phase2,Phase3,Phase4):
    ############################################

    def LatLongAlt(Traj):
        LLs = []
        for T in Traj:
            x = T[0]
            y = T[1]
            z = T[2]
            r   = np.linalg.norm(T[0:3])

            lat = np.rad2deg(np.arcsin(z/r))

            long = np.rad2deg(np.arctan2(y,x))
            if(x>0):long+=0
            elif(y>0):long+=180
            else:long-=180
            LLs.append([lat,long-80.649])
        return LLs


    def AltVelMass(Traj):
        Xs =[]
        for T in Traj:
            r   = np.linalg.norm(T[0:3])
            alt = (r-Re)*Lstar/1000
            v = np.linalg.norm(T[3:6])*Vstar/1000
            m = T[6]*Mstar
            t = T[7]*Tstar
            Xs.append([alt,v,m,t])

        return np.array(Xs).T

    LLs = LatLongAlt([Phase1[0],Phase2[0],Phase3[0],Phase4[0],Phase4[-1]])

    P1 = AltVelMass(Phase1)
    P2 = AltVelMass(Phase2)
    P3 = AltVelMass(Phase3)
    P4 = AltVelMass(Phase4)

    fig = plt.figure()
    ax0 = plt.subplot(321)
    ax1 = plt.subplot(323)
    ax2 = plt.subplot(325)
    ax3 = plt.subplot(122)

    ax0.grid(True)
    ax1.grid(True)
    ax2.grid(True)

    ax0.set_ylabel("h (km)")
    ax1.set_ylabel("v (km)/s")
    ax2.set_ylabel("M (kg)")
    ax2.set_xlabel("t (s)")

    PS = [P1,P2,P3,P4]

    for i,P in enumerate(PS):
        s = i+1
        ax0.plot(P[3],P[0])
        ax1.plot(P[3],P[1])
        ax2.plot(P[3],P[2],label=f'Stage {s}')



    m = Basemap(projection='lcc',
                lat_1=45.,lat_2=55,lat_0=50,lon_0=-65.,
                resolution=None,width=9000000,height=9000000,ax=ax3)

    for i in range(0,len(LLs)-1):
        lon1 = float(LLs[i][1])
        lat1 = float(LLs[i][0])
        lon2 = float(LLs[i+1][1])
        lat2 = float(LLs[i+1][0])
        m.drawgreatcircle(lon1=lon1,lat1=lat1,lon2=lon2,lat2=lat2)

    ax2.legend()
    #m.bluemarble()
    m.shadedrelief()

    m.drawparallels(np.arange(-90.,91.,30.))
    m.drawmeridians(np.arange(-180.,181.,60.))
    #m.drawmapboundary(fill_color='aqua')
    plt.title("Ground Track")

    fig.set_size_inches(15.0, 7.5, forward=True)
    fig.tight_layout()

    plt.show()

###############################################################################

if __name__ == "__main__":

    ast.SoftwareInfo()


    # Target orbital elements
    at     = 24361140 /Lstar
    et     = .7308
    Ot     = np.deg2rad(269.8)
    Wt     = np.deg2rad(130.5)
    istart = np.deg2rad(28.5)


    y0      = np.zeros((6))
    y0[0:3] = np.array([np.cos(istart),0,np.sin(istart)])*Re
    y0[3:6] =-np.cross(y0[0:3],np.array([0,0,We]))
    y0[3]  += 0.00001/Vstar  # cant be exactly zero,our drag equation's derivative would NAN !!!


    ## MF is the only magic number in the script, just trying to find
    ## a mean anomaly such that the terminal state on the orbit is downrange
    ## eastward from KSC in and doesnt pass through earth when LERPed from KSC
    MF   =-.05
    OEF  = [at,et,istart,Ot,Wt,MF]
    yf   = ast.Astro.classic_to_cartesian(OEF,mu)

    ts   = np.linspace(0,tf_phase4,1000)

    IG1 =[]
    IG2 =[]
    IG3 =[]
    IG4 =[]


    for t in ts:
        X = np.zeros((11))
        X[0:6]= y0 + (yf-y0)*(t/ts[-1])
        X[7]  = t
        X[8:11]= np.array([0,1,0])
        if(t<tf_phase1):
            m= m0_phase1 + (mf_phase1-m0_phase1)*(t/tf_phase1)
            X[6]=m
            IG1.append(X)
        elif(t<tf_phase2):
            m= m0_phase2 + (mf_phase2-m0_phase2)*(( t-tf_phase1) / (tf_phase2 - tf_phase1))
            X[6]=m
            IG2.append(X)
        elif(t<tf_phase3):
            m= m0_phase3 + (mf_phase3-m0_phase3)*(( t-tf_phase2) / (tf_phase3 - tf_phase2))
            X[6]=m
            IG3.append(X)
        elif(t<tf_phase4):
            m= m0_phase4 + (mf_phase4-m0_phase4)*(( t-tf_phase3) / (tf_phase4 - tf_phase3))
            X[6]=m
            IG4.append(X)


In [2]:
IG1

[array([ 8.78817113e-01,  0.00000000e+00,  4.77158760e-01,  1.50670767e-09,
         6.15850718e-02, -0.00000000e+00,  1.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  1.00000000e+00,  0.00000000e+00]),
 array([ 8.78854497e-01,  2.62712123e-04,  4.77178557e-01, -5.62161524e-04,
         6.28880999e-02, -3.07814187e-04,  9.94500909e-01,  1.00100100e-03,
         0.00000000e+00,  1.00000000e+00,  0.00000000e+00]),
 array([ 8.78891881e-01,  5.25424246e-04,  4.77198354e-01, -1.12432455e-03,
         6.41911279e-02, -6.15628374e-04,  9.89001819e-01,  2.00200200e-03,
         0.00000000e+00,  1.00000000e+00,  0.00000000e+00]),
 array([ 8.78929266e-01,  7.88136368e-04,  4.77218152e-01, -1.68648758e-03,
         6.54941560e-02, -9.23442561e-04,  9.83502728e-01,  3.00300300e-03,
         0.00000000e+00,  1.00000000e+00,  0.00000000e+00]),
 array([ 0.87896665,  0.00105085,  0.47723795, -0.00224865,  0.06679718,
        -0.00123126,  0.97800364,  0.004004  ,  0.        ,  1.        ,
  